In [ ]:
        # "org.apache.spark:spark-avro_2.13:3.4.1"

In [ ]:
from pyspark.sql import SparkSession

# -------------------
# Spark session
# -------------------
spark = (
    SparkSession.builder
        .appName("Delta S3 Hive Auto Insert")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1,"
                "org.apache.spark:spark-avro_2.13:3.4.1")  # <- add this
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO / S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate()
)


# Set logging level to ERROR to reduce warnings
spark.sparkContext.setLogLevel("ERROR")

print("Spark session started with ERROR log level")

# Check current catalog
current_catalog = spark.catalog.currentCatalog()
print("Current catalog:", current_catalog)

# Check current database
current_db = spark.catalog.currentDatabase()
print("Current database:", current_db)

In [ ]:
from pyspark.sql.functions import current_timestamp, col, input_file_name, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import StringType
from urllib.parse import urlparse
import boto3

def process_batch(df_batch, batch_id):
    # 1️⃣ Get distinct file paths for this batch
    file_paths = [row.file_path for row in df_batch.select("file_path").distinct().collect()]
    
    # 2️⃣ Get last modified timestamps
    s3_client = boto3.client(
        "s3",
        endpoint_url="http://localhost:9000",
        aws_access_key_id="admin",
        aws_secret_access_key="password"
    )
    file_ts_map = {}
    for path in file_paths:
        parsed = urlparse(path)
        bucket = parsed.netloc
        key = parsed.path.lstrip("/")
        file_ts_map[path] = s3_client.head_object(Bucket=bucket, Key=key)["LastModified"].isoformat()
    
    # 3️⃣ Apply mapping using UDF
    from pyspark.sql.functions import udf
    from pyspark.sql.types import StringType
    get_file_ts_udf = udf(lambda p: file_ts_map.get(p, None), StringType())
    
    df_flat = df_batch.select(
        "device_id",
        "record_id",
        "start_time",
        "ts",
        col("signals.ecg_i").alias("ecg_i"),
        col("signals.ecg_i_filtered").alias("ecg_i_filtered"),
        col("file_path"),
    ).withColumn("file_modified_ts", get_file_ts_udf(col("file_path"))) \
     .withColumn("ingest_ts", current_timestamp())
    
    # 4️⃣ Write to Delta
    df_flat.write.format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .save("s3a://warehouse/silver.db/silver_ecg")
    
    # 5️⃣ Delete processed files
    for path in file_paths:
        parsed = urlparse(path)
        bucket = parsed.netloc
        key = parsed.path.lstrip("/")
        s3_client.delete_object(Bucket=bucket, Key=key)

# 1️⃣ Define Avro schema
avro_schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("record_id", StringType(), True),
    StructField("start_time", StringType(), True),
    StructField("ts", DoubleType(), True),
    StructField("signals", StructType([
        StructField("ecg_i", DoubleType(), True),
        StructField("ecg_i_filtered", DoubleType(), True)
    ]), True)
])

# 2️⃣ Create streaming DataFrame
df_stream = spark.readStream.format("avro") \
    .schema(avro_schema) \
    .option("maxFilesPerTrigger", 10) \
    .load("s3a://warehouse/fs/ecg_avro/")

# 3️⃣ Add file path column
df_stream = df_stream.withColumn("file_path", input_file_name())

# Streaming write with foreachBatch
df_stream.writeStream \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", "s3a://warehouse/fs/ecg_checkpoint/_checkpoint/") \
    .start() \
    .awaitTermination()


In [ ]:
# from pyspark.sql.functions import current_timestamp, col, input_file_name, udf
# from pyspark.sql.types import StringType
# from urllib.parse import urlparse
# import boto3

# # -------------------
# # S3 UDF for file last modified
# # -------------------
# def get_file_ts(file_path):
#     s3_client = boto3.client(
#         "s3",
#         endpoint_url="http://localhost:9000",
#         aws_access_key_id="admin",
#         aws_secret_access_key="password"
#     )
#     parsed = urlparse(file_path)
#     bucket = parsed.netloc
#     key = parsed.path.lstrip("/")
#     try:
#         return s3_client.head_object(Bucket=bucket, Key=key)["LastModified"].isoformat()
#     except:
#         return None

# get_file_ts_udf = udf(get_file_ts, StringType())

# # -------------------
# # Flatten streaming DF
# # -------------------
# df_flat_stream = df_stream.select(
#     "device_id",
#     "record_id",
#     "start_time",
#     "ts",
#     col("signals.ecg_i").alias("ecg_i"),
#     col("signals.ecg_i_filtered").alias("ecg_i_filtered"),
#     get_file_ts_udf(col("file_path")).alias("file_modified_ts"),
#     current_timestamp().alias("ingest_ts")
# )

# # -------------------
# # Write streaming DF to Delta
# # -------------------
# df_flat_stream.writeStream.format("delta") \
#     .option("checkpointLocation", "s3a://warehouse/fs/ecg_checkpoint/_checkpoint/") \
#     .option("path", "s3a://warehouse/silver.db/silver_ecg") \
#     .option("mergeSchema", "true") \
#     .outputMode("append") \
#     .start() \
#     .awaitTermination()
